# Time to 3D rendering
Converts the video from an Echography to a 3D rendering where time becomes the 3D dimension
Uses VTK marching cubes algorithm
Renders the 3D view with Plotly

In [1]:
# VTK and CV2 imports 
import vtk
import numpy as np
from vtk.util import numpy_support,vtkImageImportFromArray
from vtkmodules.vtkCommonDataModel import vtkImageData
from vtkmodules.vtkFiltersCore import (
    vtkFlyingEdges3D,
    vtkMarchingCubes
)
from vtkmodules.vtkFiltersSources import vtkSphereSource
from vtkmodules.vtkIOImage import vtkDICOMImageReader
from vtkmodules.vtkImagingHybrid import vtkVoxelModeller
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)
from vtkmodules.vtkCommonColor import vtkNamedColors
import vtkmodules.all as vtk
import cv2
# read video file
# Sample anonymized echocardiography data (replace with your actual data)
videofile="./Anonymized_EchoCardiography06.mp4"
print("Read",videofile)
video = cv2.VideoCapture(videofile)
anonframes=[]
print("video opened",video.isOpened())
while(video.isOpened()):
    ret, frame = video.read()
    if ret == True:       
        anonframe= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
        anonframes.append(anonframe)
    else:
        break
# stack frames from video to create the 3D rendering
image_data=np.stack(anonframes)
# Create a VTK image importer
image_importer = vtkImageImportFromArray.vtkImageImportFromArray()
# Set the array data to the importer
image_importer.SetArray(image_data)

colors = vtkNamedColors()

# Update the importer
image_importer.Update()

# Prepare the Marching Cubes algorithm setting the grayscale luminosity threshold to 127 
# Rendering to triangles only occurs with a formal rendering call
surface = vtkMarchingCubes()
surface.SetInputData(image_importer.GetOutput()) 
surface.ComputeNormalsOn()
surface.SetValue(0,127)

# create a renderer
renderer = vtkRenderer()
renderer.SetBackground(colors.GetColor3d('DarkSlateGray'))

render_window = vtkRenderWindow()
render_window.AddRenderer(renderer)
render_window.SetWindowName('MarchingCubes')

interactor = vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# create a Polygone data mapper
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(surface.GetOutputPort())
mapper.ScalarVisibilityOff()

# Add the mapper as actor to the renderer
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('MistyRose'))
renderer.AddActor(actor)


Read ./Anonymized_EchoCardiography06.mp4
video opened True


In [2]:
# do not use VTK screens to render, but generate the polygons
render_window.SetOffScreenRendering(True)
render_window.Render()

In [3]:
# Get the cell polygons (triangles) from the rendered surface
from vtk.util.numpy_support import vtk_to_numpy
polydata=surface.GetOutput()
cells = polydata.GetPolys()
nCells = cells.GetNumberOfCells()
array = cells.GetData()
# This holds true if all polys are of the same kind, e.g. triangles.
assert(array.GetNumberOfValues()%nCells==0)
nCols = array.GetNumberOfValues()//nCells
numpy_cells = vtk_to_numpy(array)
numpy_cells = numpy_cells.reshape((-1,nCols))
print(numpy_cells.T.shape)

(4, 2754874)


In [4]:
# get the points 
points = polydata.GetPoints()
pdata=points.GetData()
numpy_points = vtk_to_numpy(pdata)
print(numpy_points.T.shape)

(3, 1388585)


In [ ]:
import plotly.graph_objects as go
# transpose for use with Plotly
points=numpy_points.T
# first cells columns is the size of the polygon (we skip it as it is always 3 with marching cubes)
cells=(numpy_cells.T)[1:]

fig = go.Figure(data=[
    go.Mesh3d(
        #  vertices of a cube
        x=points[2],
        y=points[0],
        z=np.amax(points[1])-points[1],
        colorbar_title='Point #',
        colorscale=[[0, 'gold'],
                    [0.2, 'mediumturquoise'],
                    [1, 'magenta']],
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity = np.linspace(0, 1, 1388585, endpoint=True),
        # i, j and k give the vertices of triangles
        i = cells[0],
        j = cells[1],
        k = cells[2],
        name='y',
        showscale=True
    )
])
fig.update_layout(
    scene=dict(
        xaxis=dict( title='Echography Frame #'),
        yaxis=dict(title='width'),
        zaxis=dict(title='height'),
    )
)
fig.update_layout(
    title=dict(text="Time to 3D, Use mouse to rotate, zoom, etc.")
)
# save html and show
fig.write_html("/mnt/d/html/vtk3d.html")
fig.show()